# Analyse OSISAF (https://osi-saf.ifremer.fr) data inside DATARMOR and outside DATARMOR (https access) 

Example of opening file direct from a netcdf file it self.  

In [1]:
import fsspec
import xarray as xr
url='https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0000Z.nc'
fs = fsspec.filesystem('https')
ds = xr.open_dataset(fs.open(url))
ds.H0.min().compute()
#xr.open_dataset('https://data-dataref.ifremer.fr/marc/f1_e2500_agrif/MARC_F1-MARS3D-SEINE/best_estimate/2018/MARC_F1-MARS3D-SEINE_20180801T0000Z.nc')

<xarray.DataArray 'H0' ()>
array(-6.8009653, dtype=float32)

In [11]:
import aiohttp
#https://osi-saf.ifremer.fr/
user='c1f2a8'
password='xxx'
fs = fsspec.filesystem('https',
                       client_kwargs = {'auth': aiohttp.BasicAuth(user, password),
                       #'allow_redirects': True
                                       }
                      )

This works on datarmor: 
'''
qsub -I -q mpi_1 -l walltime=1:00:00
bash
conda activate /home/datawork-lops-iaocea/conda-env/pangeo-fish_1222/
cd /home/datawork-taos-s/test
ipython
'''

In [1]:
import fsspec
import intake
import xarray as xr

catalogue="/home/datawork-taos-s/intranet/kerchunk/ref-marc.yaml"
catalogue = "https://data-taos.ifremer.fr/kerchunk/ref-marc.yaml"

Detail of the imports:
-----------------------------------------------------------------------------------------------  
    Intake    
>- Handle the catalog contained in the yaml files.

    dask_hpcconfig    
>- Parrallelize your tasks on an hpc like datatrmor.

    Distributed import Client    
>- Client is creating the dask client

    Xarray    
>- Xarray makes operation on datasets and creates labbeled multidimensional arrays

    hvplot.xarray
>- Plotting 

    geoviews.features
>- Plotting that enables some features on maps 

    os
>- Can handle operating system dependant functionnality 

In [2]:
def list_param(cat, param):
    return cat.metadata["parameters"][param]["allowed"]

def allowed_param(cat):
    return cat.metadata["parameters"]

def allowed_years(region):
    fs = fsspec.filesystem("https") if ('https' in catalogue) else fsspec.filesystem("file")
    files = fs.glob(catalogue.replace(".yaml", "/") + region + "/*.json.zst")
    years = [
        f.replace(".json.zst", "").replace(
            catalogue.replace(".yaml", "/") + region + "/", ""
        )
        for f in files
    ]
    years.sort()
    return years

def year_concat(cat, source, region):
    concat = [
        cat[source](region=region, year=i).to_dask() for i in allowed_years(region)
    ]

    return xr.concat(
        concat, dim="time", coords="minimal", data_vars="minimal", compat="override"
    )

Detail of the functions:
-----------------------------------------------------------------------------------------------  
    list_param(cat, param)    
>- Takes 2 arguments: cat = An opened catalog, param = The name of a parameter.
>- Returns a ptyhon list containing all the parameters.

    allowed_param(cat)    
>- Takes 1 argument: cat = An opened catalog
>- Returns a python dict containing the allowed values of each parameters and their description

    Allowed_years(region)    
>- Takes 1 argument: region = A region of data.
>- Returns a pythton list containing all the year available in kerchunk catalogue.  

    year_concat(cat, source, region)    
>- Takes 3 arguments: cat = An opened catalog, source = the source from this catalog, region = the region where the data comes from
>- Returns: A dataset concatenated over the all available in kerchunk. 


# Open intake catalgue.  

In [3]:
cat = intake.open_catalog(catalogue)

## Inspect intake catalogue

In [45]:
regions = list_param(cat, "region")

In [8]:
ds = cat.marc(region=region, year="2010").to_dask()

In [46]:
regions

['f1_e2500_agrif/MARC_F1-MARS3D-ADOUR',
 'f1_e2500_agrif/MARC_F1-MARS3D-ARMOR',
 'f1_e2500_agrif/MARC_F1-MARS3D-FINIS',
 'f1_e2500_agrif/MARC_F1-MARS3D-GIRONDE',
 'f1_e2500_agrif/MARC_F1-MARS3D-LOIRE',
 'f1_e2500_agrif/MARC_F1-MARS3D-MANGAE2500-AGRIF',
 'f1_e2500_agrif/MARC_F1-MARS3D-MORBIHAN',
 'f1_e2500_agrif/MARC_F1-MARS3D-PDC',
 'f1_e2500_agrif/MARC_F1-MARS3D-SEINE',
 'l1_v9/MARC_L1-MARS2D-AQUI250',
 'l1_v9/MARC_L1-MARS2D-ATLNE2000',
 'l1_v9/MARC_L1-MARS2D-FINIS250',
 'l1_v9/MARC_L1-MARS2D-MANE250',
 'l1_v9/MARC_L1-MARS2D-MANGA700',
 'l1_v9/MARC_L1-MARS2D-MANW250',
 'l1_v9/MARC_L1-MARS2D-SUDBZH250',
 'b1',
 'b1_v10',
 'b1_v10r2019',
 'f1_e2500',
 'f1_e4000',
 'f2_1200',
 'f2_1200_sn',
 'f2_1200_v10',
 'l1_brest',
 'l2_v10',
 'ww3/ANTILLES-3MIN',
 'ww3/ANTILLES-UG',
 'ww3/ATNE-10M',
 'ww3/FINIS-200M',
 'ww3/GLOBAL-30MIN',
 'ww3/MED-6MIN',
 'ww3/MENOR-2MIN',
 'ww3/MENOR-UG',
 'ww3/NORGAS-2MIN',
 'ww3/NORGAS-UG',
 'ww3/NORGAS-UG_V3',
 'ww3/REUNION-UG']

## Define a region, inspect the catalogue to know which year the datas are availableand, then load the data

In [4]:
region = "f1_e2500_agrif/MARC_F1-MARS3D-SEINE"

In [5]:
allowed_years(region)

['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

## Load and concartenate all existing year

In [37]:
%%time
ds = year_concat(cat, "marc", region)

CPU times: user 15.2 s, sys: 2.67 s, total: 17.8 s
Wall time: 10min 55s


## Plot data

In [50]:
%%time
da = ds.XE.unify_chunks()
da

CPU times: user 1.65 ms, sys: 79 µs, total: 1.73 ms
Wall time: 1.75 ms


<xarray.DataArray 'XE' (time: 8760, nj: 183, ni: 413)>
dask.array<open_dataset-60e4c57cdd6bf9f8f8f03fa5492e51f2XE, shape=(8760, 183, 413), dtype=float32, chunksize=(1, 183, 413), chunktype=numpy.ndarray>
Coordinates:
    latitude   (nj, ni) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
    longitude  (nj, ni) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
  * ni         (ni) float32 0.0 1.0 2.0 3.0 4.0 ... 409.0 410.0 411.0 412.0
  * nj         (nj) float32 0.0 1.0 2.0 3.0 4.0 ... 179.0 180.0 181.0 182.0
  * time       (time) datetime64[us] 2018-01-01 ... 2018-12-31T23:00:00
Attributes:
    long_name:      sea surface height
    standard_name:  sea_surface_height_above_geoid
    units:          m
    valid_max:      50.0
    valid_min:      -50.0

In [13]:
#import geoviews.feature as gf
import hvplot.xarray

da.hvplot.quadmesh(
    x="longitude", y="latitude", rasterize=True, geo=True, cmap="BuPu", coastline=True
)
# ) * gf.land.opts(scale="10m") * gf.coastline.opts(scale="10m")

:DynamicMap   [time]
   :Overlay
      .Image.I     :Image   [longitude,latitude]   (XE)
      .Coastline.I :Feature   [Longitude,Latitude]

### Annex: Tests on concartenations. 

Tested on datarmor with one full node 


|    REGION                               |  OPEN DATASET  |      CONCAT     |   COMPUTE   |     PLOT      |Concat time (s) |Nbr of Years |
|    :------------------------------------|:--------------:|:---------------:|:-----------:|:-------------:|:--------------:|:-----------:|
|   ANTILLES-UG                           |Works           |Missing variable |             |               |                |             |
|   ATNE-10M                              |Works           |Missing variable |             |               |                |             |
|   CALEDONIE-3MIN                        |Works           |Missing variable |             |               |                |             |
|   GLOBAL-30MIN                          |Works           |Missing variable |             |               |                |             |
|   MARC_F1-MARS3D-ADOUR                  |Works           |Works            |Works        |Works          |23.6            |9            |
|   MARC_F1-MARS3D-ARMOR                  |Works           |Works            |Works        |Works          |23              |9            |
|   MARC_F1-MARS3D-FINIS                  |Works           |Works            |Works        |Works          |20.9            |9            |
|   MARC_F1-MARS3D-GIRONDE                |Works           |Works            |Works        |Works          |21.1            |9            |
|   MARC_F1-MARS3D-LOIRE                  |Works           |Works            |Works        |Works          |20              |9            |
|   MARC_F1-MARS3D-MANGAE2500-AGRIF       |Works           |Works            |Works        |Works          |563 (ms)        |2            |
|   MARC_F1-MARS3D-MORBIHAN               |Works           |Works            |Works        |Works          |17              |6            |
|   MARC_F1-MARS3D-PDC                    |Works           |Works            |Works        |Works          |117 (1.57 s)    |9            |
|   MARC_F1-MARS3D-SEINE                  |Works           |Works            |Works        |Works          |29.8            |9            |
|   MED-6MIN                              |Works           |Missing variable |             |               |                |12           |
|   MENOR-2MIN                            |Works           |Missing variable |             |               |                |12           |
|   MENOR-UG                              |Works           |Diff  dims size  |             |               |                |12           |
|   NORGAS-2MIN                           |Works           |Missing variable |             |               |                |12           |
|   NORGAS-UG                             |Works           |Missing variable |             |               |                |15           |
|   NORGAS-UG_V3                          |Works           |Works            |Works        |               |20.5            |12           |
|   POLYNESIE-3MIN                        |Works           |Missing variable |             |               |                |             |
|   REUNION-180M                          |Works           |Works            |Works        |Works          |453 (ms)        |2            |
|   REUNION-UG                            |Works           |Missing variable |             |               |                |12           |
|   b1                                    |Works           |Works            |Works        |fails          |1.95            |5            |
|   b1_v10                                |Works           |Missing variable |             |               |                |11           |
|   b1_v10r2019                           |Works           |Works            |Works        |Works          |2.43            |11           |
|   f1_e2500                              |Works           |Works            |Works        |Works          |127 (2m07)      |11           |
|   f1_e4000                              |Works           |Works            |Works        |Works          |128 (2m08)      |17           |
|   f2_1200                               |Works           |Works            |Works        |Works          |23.4            |10           |
|   f2_1200_sn                            |Works           |Works            |Works        |Works          |2.84            |4            |
|   f2_1200_v10                           |Works           |Works            |Works        |Works          |18.2            |8            |
|   l2_v10                                |Works           |Works            |Works        |Works          |8.17            |9            |
|                                         |                |                 |             |               |                |             |


In [9]:
ds

<xarray.Dataset>
Dimensions:      (level: 40, nj: 183, ni: 413, nj_u: 183, ni_u: 413, nj_v: 183,
                  ni_v: 413, time: 8759, ni_f: 413, nj_f: 183)
Coordinates: (12/16)
    latitude     (nj, ni) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
    latitude_u   (nj_u, ni_u) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
    latitude_v   (nj_v, ni_v) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
  * level        (level) float32 -0.9875 -0.9625 -0.9375 ... -0.0375 -0.0125
    longitude    (nj, ni) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
    longitude_u  (nj_u, ni_u) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
    ...           ...
  * ni_v         (ni_v) float32 0.0 1.0 2.0 3.0 4.0 ... 409.0 410.0 411.0 412.0
  * nj           (nj) float32 0.0 1.0 2.0 3.0 4.0 ... 179.0 180.0 181.0 182.0
  * nj_f         (nj_f) float32 0.5 1.5 2.5 3.5 4.5 ... 179.5 180.5 181.5 182.5
  * nj_u         (nj_u) float32 0.0 1.0 2.0 3.0 4.0 ... 179.0 180.0 181.0 182.0
  * nj_v         (nj_v) float32 0.5 1.5 2.5 3.5 4.5 ... 179.5 180.5 181.5 182.5
  * time         (time) datetime64[us] 2010-01-01T01:00:00 ... 2010-12-31T23:...
Data variables: (12/15)
    Csu_sig      (level) float32 dask.array<chunksize=(40,), meta=np.ndarray>
    H0           (nj, ni) float32 dask.array<chunksize=(183, 413), meta=np.ndarray>
    HX           (nj_u, ni_u) float32 dask.array<chunksize=(183, 413), meta=np.ndarray>
    HY           (nj_v, ni_v) float32 dask.array<chunksize=(183, 413), meta=np.ndarray>
    SAL          (time, level, nj, ni) float32 dask.array<chunksize=(1, 28, 129, 290), meta=np.ndarray>
    SIG          (level) float32 dask.array<chunksize=(40,), meta=np.ndarray>
    ...           ...
    V            (time, nj_v, ni_v) float32 dask.array<chunksize=(1, 183, 413), meta=np.ndarray>
    VZ           (time, level, nj_v, ni_v) float32 dask.array<chunksize=(1, 28, 129, 290), meta=np.ndarray>
    XE           (time, nj, ni) float32 dask.array<chunksize=(1, 183, 413), meta=np.ndarray>
    b            float32 ...
    hc           (nj, ni) float32 dask.array<chunksize=(183, 413), meta=np.ndarray>
    theta        float32 ...
Attributes: (12/54)
    CDI:                       Climate Data Interface version 1.5.0 (http://c...
    CDO:                       Climate Data Operators version 1.5.0 (http://c...
    Conventions:               CF-1.6 OCO-1.3.1 COMODO-1.0
    NCO:                       4.0.6
    arakawa_grid_type:         C1
    area:                      SEINE
    ...                        ...
    references:                http://www.previmer.org/
    run_time:                  2015-11-27T08:45:23Z
    software_version:          PREVIMER forecasting system v2
    southernmost_latitude:     49.2424
    title:                     PREVIMER F1 MANGAE2500 AGRIF hindcast
    westernmost_longitude:     -1.4598